In [ ]:
!pip install transformers
!pip install langchain
!pip install chromadb
!nvidia-smi
!pip install angle-emb
!pip install pypdf

In [1]:
import torch                                       #deep learning
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
import chromadb
from langchain.chat_models import ChatOpenAI

In [2]:
from angle_emb import AnglE, Prompts
angle = AnglE.from_pretrained('WhereIsAI/UAE-Large-V1', pooling_strategy='cls').cuda()
angle.set_prompt(prompt=Prompts.C)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
from langchain.prompts import PromptTemplate

# PROMPT_TEMPLATE = """
# Answer the question based only on the following context:
# {context}
# Answer the question based on the above context: {question}.
# Provide a detailed answer.
# Don’t justify your answers.
# Don’t give information not mentioned in the CONTEXT INFORMATION.
# Do not say "according to the context" or "mentioned in the context" or similar.
# """



In [4]:
DOC_PATH = "/content/script.pdf"
CHROMA_PATH = "your_db_name"

# ----- Data Indexing Process -----

# load your pdf doc
loader = PyPDFLoader(DOC_PATH)
pages = loader.load()

# split the doc into smaller chunks i.e. chunk_size=500
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(pages)


In [5]:
whole_script=[chunks[i].page_content for i in range(len(chunks))]

In [6]:
client = chromadb.PersistentClient(path="/content/here")
client = client.get_or_create_collection(name="test")

In [ ]:
inp=[{'text': text} for text in whole_script]
vecs = angle.encode(inp, to_numpy=True)
# print(len(vecs))
ids=["text_"+str(j+1) for j in range(len(whole_script))]
print(ids)
client.add(
    embeddings = vecs,
    documents = whole_script,
    ids = ids)

In [ ]:
!pip install autoawq

In [ ]:
!pip install optimum

In [ ]:
!pip install auto-gptq

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, MusicgenForConditionalGeneration

model_name_or_path = "TheBloke/zephyr-7B-beta-GPTQ"
# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"
# model = AutoModelForCausalLM.from_pretrained(model_name_or_path,device_map="auto")

# tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

In [9]:
model_id= "TheBloke/zephyr-7B-beta-GPTQ"

In [10]:
def retrive_topK(query=[],topk=5):
    client = chromadb.PersistentClient(path="/content/here")
    here = client.get_or_create_collection(name="test")
    inp=[{'text': text} for text in query]
    query_embeddings = angle.encode(inp, to_numpy=True)
    results = here.query(
        query_embeddings=query_embeddings,
        n_results=topk
    )
    return results['documents']

In [11]:
# Define prompt template
PROMPT_TEMPLATE ="""
<|system|>
You are a great AI which can retrive answer from a given context very concisely. If the answer is not explicitly present in the context, you need to provide a summarization of the points referencing the answer mentioned in the context.
</s>

<|user|>
Answer the question based only on the following context:
{context}
Answer the question based on the above context: {question}.

</s>

<|assistant|>
"""

prompt_template = PromptTemplate(input_variables=["context", "question"], template=PROMPT_TEMPLATE)

In [12]:
def generate_answer_from_context_and_query(prompt_template, model_id, context_text, query):
    # tokenizer = AutoTokenizer.from_pretrained(model_id)
    # model = AutoModelForCausalLM.from_pretrained(model_id, load_in_8bit=False, max_length=600)
    model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
    input_text = prompt_template.format(context=context_text, question=query)
    inputs = tokenizer(input_text, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=512)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
prompt_template

In [15]:
context_text = retrive_topK(["Explain the theme of the movie?"])
query = "Explain the theme of the movie?"
# print(context_text)

In [19]:
# Generate and print answer based on context and query
answer = generate_answer_from_context_and_query(prompt_template, model_id, context_text, query)
print(answer)

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4193: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1363: UserWarning: Current model requires 1042646016 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1477: UserWarning: You are calling


<|system|>
You are a great AI which can retrive answer from a given context very concisely. If the answer is not explicitly present in the context, you need to provide a summarization of the points referencing the answer mentioned in the context.
 

<|user|>
Answer the question based only on the following context:
[['RICK DECKARD / SUBJECT # N6RRP40619.Up comes A EXTREME-CLOSE IMAGE OF...AN EYELARGE.  LOVELY.  A rich green we so far haven’t seen in the film.  AN OLD VOIGHT-KAMPFF RECORDING.  The eye BLINKS.Over it comes a JUMBLE OF VOICES, falling in and out, incomplete audio.  YET ALL HAUNTINGLY FAMILIAR.ON K.  WATCHING, AS --MAN’S VOICE (O.S.)Have a little boy--butterfly collection plus the killing----Feel a wasp crawling on your wrist----Just answer the question--WOMAN’S VOICE (O.S.)--I’d take him to the', 'of dust.  Death creeping in.  Nearly there.A thin SOUND in the distance.  FOOTSTEPS.  Feet pass Deckard’s blaster covered in ash.  A beat.THEN K IS DRAGGED BACKWARDS ACROSS THE 

**Explain the theme of the movie?**
1. Identity and memory: The recurring images of eyes and memories, as well as the use of the Voight-Kampff test to distinguish replicants from humans, suggest a focus on identity and memory.
2. Mortality and the human condition: The scenes of death and decay, as well as the references to pain and fragility, suggest a contemplation of mortality and the human condition.
3. Technology and its impact on society: The presence of advanced technology, such as replicants and the Spinner, suggests a commentary on the role of technology in society and its potential consequences.
4. Deception and perception: The use of deception and misdirection, such as the false leads and hidden agendas, suggests a focus on deception and perception.
5. Justice and morality: The conflict between Deckard and the replicants raises questions about justice and morality, particularly in a society where replicants are created to serve humans.

However, without a complete viewing of the movie, it is difficult to determine the exact theme or themes that the film explores.

In [14]:
context_text = retrive_topK(["Who are the characters? "])
query = "Who are the characters?"

answer = generate_answer_from_context_and_query(prompt_template, model_id, context_text, query)
print(answer)

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4193: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1363: UserWarning: Current model requires 1042646016 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1477: UserWarning: You are calling


<|system|>
You are a great AI which can retrive answer from a given context very concisely. If the answer is not explicitly present in the context, you need to provide a summarization of the points referencing the answer mentioned in the context.
 

<|user|>
Answer the question based only on the following context:
[['K (V.O.)I go looking for a hiding place.  There’s nowhere to go but this... dark furnace.  It’s very dark.  I’m very scared...  But this horse is all I have so I go in anyway.A FURNACE RAGES WITH FIRE.  He moves to THE DARK PLACES BEHIND.  The child’s hand opens a secret space in the back.THE LARGER BOYS FIND THE CHILD IN FRONT OF THE FLAMES.K (V.O.)They find me and beat me to tell them where is it.  But I don’t.INT. K’S APARTMENT. RESUME SCENE.BACK TO:  K.  His eyes open.  The spell of memory', 'in darkness.Seeing K, the CHILDREN stop still.  Eyes wide and astonished.  If they’ve ever seen anyone like him it wasn’t lately.They circle K like he was a newfound species.  Fa

**Who are the characters?**

Based on the given context, the characters mentioned are:

1. K (V.O.)
2. Child
3. Larger boys
4. Children (plural)
5. Old woman
6. Neighbors (plural)
7. Boy (brown-faced, missing an ear)
8. Luv (presumably a person)
9. Barracudas (plural)
10. Niander Wallace (presumably a person)

Note: The context provided does not give a clear indication of the relationships between these characters, so it is unclear if they are all separate individuals or if some are related in some way.

In [13]:
context_text = retrive_topK(["How many male and female characters are in the movie?"])
query = "How many male and female characters are in the movie?"

answer = generate_answer_from_context_and_query(prompt_template, model_id, context_text, query)
print(answer)

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4193: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1363: UserWarning: Current model requires 1042646016 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1477: UserWarning: You are calling


<|system|>
You are a great AI which can retrive answer from a given context very concisely. If the answer is not explicitly present in the context, you need to provide a summarization of the points referencing the answer mentioned in the context.
 

<|user|>
Answer the question based only on the following context:
[['RICK DECKARD / SUBJECT # N6RRP40619.Up comes A EXTREME-CLOSE IMAGE OF...AN EYELARGE.  LOVELY.  A rich green we so far haven’t seen in the film.  AN OLD VOIGHT-KAMPFF RECORDING.  The eye BLINKS.Over it comes a JUMBLE OF VOICES, falling in and out, incomplete audio.  YET ALL HAUNTINGLY FAMILIAR.ON K.  WATCHING, AS --MAN’S VOICE (O.S.)Have a little boy--butterfly collection plus the killing----Feel a wasp crawling on your wrist----Just answer the question--WOMAN’S VOICE (O.S.)--I’d take him to the', 'and the second back off, put off.  Punk Doxie (MARIETTE) doesn’t seem to mind.  She gives her most inviting grin.  But K’s not interested.MARIETTE (cont’d)Buy a lady a cigarette

**How many male and female characters are in the movie?**

Based on the given context, it is unclear how many male and female characters are in the movie. The context only provides brief snippets of dialogue and actions, and does not provide a comprehensive list of all the characters in the movie.

In [13]:
context_text = retrive_topK(["Does the script pass the Bechdel test?"])
query = "Does the script pass the Bechdel test?"

answer = generate_answer_from_context_and_query(prompt_template, model_id, context_text, query)
print(answer)

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4193: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1363: UserWarning: Current model requires 1042646016 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1477: UserWarning: You are calling


<|system|>
You are a great AI which can retrive answer from a given context very concisely. If the answer is not explicitly present in the context, you need to provide a summarization of the points referencing the answer mentioned in the context.
 

<|user|>
Answer the question based only on the following context:
[['(cont’d)Run it.ANOTHER CHIME.  “ALL EXEGETIC DATA CORRUPTED.”  Annoyed --K (cont’d)Ok.  Then run it raw.  On a rondo.The SLATE recedes.  A FAN begins to WHIR as a VIEWING DEVICE comes to life.  Like an old moviola.  Blinders on each side.  K situates himself in front of it -- CLICKS on the emanator -- and cranks the machine.A DATAFLOW begins on its SCREEN.  A CASCADE OF ONLY FOUR LETTERS:  A-C-T-G.  RAW GENETIC CODE.  WITH NO IDENTIFYING  CONTEXTUAL DATA TO LINK WITH SPECIFIC TRAITS.  ONLY RAW', 'He quiets.  Just then -- K sees something in the dataflow.  Sits up suddenly.K (cont’d)There.  Hold.The snowstorm of LETTERS FREEZES.  K moves close.  Sees something remarkable i

**Does the script pass the Bechdel test?**

No, the script does not pass the Bechdel test as there are no conversations between two named female characters about a topic other than a man. The only female character, LUV, interacts with a male character, K, and their conversation is about K's job and a DNA test. There is also a female voice giving instructions, but it is not clear if she is a named character. Therefore, the script fails the Bechdel test.

In [13]:
context_text = retrive_topK(["What is the role of Deckard in the movie?"])
query = "What is the role of Deckard in the movie?"

answer = generate_answer_from_context_and_query(prompt_template, model_id, context_text, query)
print(answer)

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4193: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1363: UserWarning: Current model requires 1042646016 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1477: UserWarning: You are calling


<|system|>
You are a great AI which can retrive answer from a given context very concisely. If the answer is not explicitly present in the context, you need to provide a summarization of the points referencing the answer mentioned in the context.
 

<|user|>
Answer the question based only on the following context:
[['it.  So lost in the discovery, he is completely surprised by -- THE CLICK OF A GUN --MAN’S VOICE (O.S.)“You mightn’t happen to have a piece of cheese about you now, boy?”DECKARD’S ICONIC BLASTERComes out of the SHADOWS half into the LIGHT.  PrecedingRICK DECKARDStrong, tireless.  Seasoned by time and, like teak or copper, far more handsome for the patina.  But also something... cracky about him.  Thoughts obscure, running together.  Enough to keep us guessing if he’s gone touched left alone out', 'as --ONE OF THE DARK SPINNERS LANDS INSIDE THE PENTHOUSE.DARK FIGURES get out and DRAG UNCONSCIOUS DECKARD toward the Spinner.  K tries to move for Deckard -- but his abdomen ha

**What is the role of Deckard in the movie?**

The question asks about the role of Deckard in the movie based on the given context. From the context provided, it can be inferred that Deckard is the main protagonist of the movie. He is described as a seasoned and experienced character, who is strong, tireless, and handsome for his patina. He is also referred to as Rick Deckard, and his iconic blaster is mentioned. Throughout the context, Deckard is involved in various actions, such as fighting his captors, helping K back, and reaching out to save K's hand. His role in the movie seems to be that of a renegade blade runner, who is trying to save K and meet his daughter. Therefore, the answer to the question would be that Deckard is the main protagonist of the movie.